In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')
if download_last_date == df_adjOHLCV_last_date:
  msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
  raise SystemExit(msg_stop )

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-02-14
df_adjOHLCV_last_date: 2023-02-13


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-02-16   df_Close:  134.26659     Yahoo_Close:  134.26659   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-02-16   df_Close:  377.00046     Yahoo_Close:  377.00046   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-02-16   df_Close:  439.47031     Yahoo_Close:  439.47034   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
# TODO add list of SPY, VGT, FTSM, QQQ, BNDX 
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'BNDX', 'USO', 'JJC', 'BCI', 'NG']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1314
symbols added:   10
unique symbols:  1318


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[]]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], []]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

symbols_download_err: [[], [], []]
downloaded symbols from chuck 2, sleep start
downloaded symbols from chuck 2, sleep ends
[*********************100%***********************]  118 of 118 completed

symbols_download_err: [[], [], [], []]
downloaded symbols from all chucks
symbols_download_err: [[], [], [], []]


In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

[]

In [9]:
if l_symbols_err:  # re-download symbols with download error 
  symbols_download_err = []
  # took 24 minutes to download 1917 symbols without error caused by Yahoo
  # for i, symbols in enumerate(symbols_chunks):
  for i, symbols in enumerate(l_symbols_err):
    df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
    symbols_download_err.append(list(shared._ERRORS.keys()))
    # print(f'symbols_download_err: {symbols_download_err}')
    print(f'\nsymbols_download_err: {symbols_download_err}')  
    df_list.append(df)
    # pause between download
    if i < len(symbols_chunks) - 1 :  # skip pause after last download
      print(f'downloaded symbols from chuck {i}, sleep start')
      # sleep 78(18m 25s)
      time.sleep(78)
      print(f'downloaded symbols from chuck {i}, sleep ends')
    else:
      print(f'downloaded symbols from all chucks')

  print(f'symbols_download_err: {symbols_download_err}')

In [10]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [11]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [12]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [13]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [14]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [15]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [16]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [17]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [18]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [19]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15386 entries, 1962-01-02 to 2023-02-14
Columns: 7908 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7882), int64(26)
memory usage: 928.4 MB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15386 entries, 1962-01-02 to 2023-02-14
Columns: 6590 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(6564), int64(26)
memory usage: 773.7 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15386 entries, 1962-01-02 to 2023-02-14
Columns: 1318 entries, A to ZWS
dtypes: float64(1318)
memory usage: 154.8 MB
df_close.info():
None

len(symbols_df):
1318



In [20]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [21]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1318
count_df_close_cols: 1318
count_cols_df_OHLCVA_downloaded: 7908
count_cols_df_adjOHLCV: 6590


In [22]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-02-16   df_Close:  134.26659     Yahoo_Close:  134.26659   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-02-16   df_Close:  377.00046     Yahoo_Close:  377.00046   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-02-16   df_Close:  439.47034     Yahoo_Close:  439.47037   %_dif_Close: 0.00001
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [23]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-14,397.000000,397.339996,382.779999,390.600006,389.049408,1709900.0
2022-02-15,387.179993,388.920013,380.299988,384.769989,383.242523,787600.0
2022-02-16,384.989990,394.260010,384.799988,393.660004,392.097260,1086100.0
2022-02-17,392.769989,401.359985,391.410004,398.109985,396.529572,887800.0


In [24]:
df_adjOHLCV.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-02-08  153.639999  155.169998  153.229996  154.419998  1073500.0   
2023-02-09  155.669998  156.320007  151.539993  152.380005   852200.0   
2023-02-10  151.800003  152.990005  151.070007  152.550003   934800.0   
2023-02-13  153.270004  154.660004  152.009995  154.600006   925400.0   
2023-02-14  154.339996  155.749893  151.449997  152.970001   647510.0   

                   AA                                            ...  \
                 Open       High        Low      Close   Volume  ...   
Date                                                             ...   
2023-02-08  52.180000  52.680000  50.610001  50.619999  4649500  ...   
2023-02-09  51.470001  51.669998  49.714001  50.009998  4197600  ...   
2023-02-10  49.279999  49.490002  48.189999  48.689999  4611000  ...   
2023-02-13  48.389999  48.700001  47.119999  47.950001  3334100  ...   
2023-02-14  47.500000  49.080002  46.939999  48.930000  4181533  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-02-08  26.290001  26.420000  25.530001  25.850000  347400.0  25.100000   
2023-02-09  26.299999  26.670000  25.040001  25.250000  326500.0  24.440001   
2023-02-10  25.030001  25.299999  24.690001  25.080000  251900.0  23.490000   
2023-02-13  25.150000  25.780001  24.709999  25.610001  322500.0  23.219999   
2023-02-14  25.219999  25.639999  24.500000  24.830000  338427.0  23.230000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-02-08  25.100000  23.570000  24.090000  4484900.0  
2023-02-09  24.570000  23.600000  23.780001  2363500.0  
2023-02-10  23.600000  22.990000  23.250000  1318500.0  
2023-02-13  23.559999  23.170000  23.500000   944700.0  
2023-02-14  24.094999  23.120001  23.889999  1985870.0  

[5 rows x 6590 columns]

In [25]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-02-08,456.059998,225.699997,89.010002,469.649994,170.350006,174.559998
2023-02-09,453.850006,220.449997,86.739998,470.089996,168.339996,173.479996
2023-02-10,463.859985,221.330002,88.239998,480.829987,170.970001,173.199997
2023-02-13,464.279999,218.940002,90.809998,481.970001,171.389999,174.220001
2023-02-14,467.510010,216.190002,90.489998,481.869995,170.300003,174.949997


In [26]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-02-08,148.740005,81.980003,23.440001
2023-02-09,147.479996,82.180000,23.309999
2023-02-10,151.970001,83.839996,23.860001
2023-02-13,147.800003,83.470001,23.809999
2023-02-14,148.789993,83.870003,23.600000


In [27]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,BNDX,USO,JJC,BCI,NG
Date,,,,,,,,,,
2023-02-08,410.649994,364.929993,107.970001,304.369995,41.259998,48.560001,68.680000,20.559999,21.170000,6.10
2023-02-09,407.089996,363.390015,107.430000,301.679993,41.419998,48.470001,67.919998,20.674999,21.059999,6.08
2023-02-10,408.040009,361.079987,106.739998,299.700012,41.070000,48.349998,69.870003,20.430000,21.290001,6.05
2023-02-13,412.829987,367.220001,108.620003,304.500000,41.439999,48.430000,69.500000,20.665001,21.260000,6.03
2023-02-14,412.640015,369.170013,109.230003,306.750000,41.320000,48.285000,69.400002,20.812300,21.299999,6.04


In [28]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-02-14,134.326202,377.068665,432.011322
2022-02-15,135.021637,372.440186,438.978333
2022-02-16,134.266586,377.000458,439.470337
2022-02-17,129.398605,378.822571,430.082642
2022-02-18,131.186859,376.571716,427.297791
